## Project 1

In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sns

%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()

True

### Current price of shares

In [3]:
# Set current amount of shares
my_jpm = 50
my_jnj = 50
my_msft = 50
my_rio = 50
my_pfe = 50

In [4]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [5]:
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [6]:
# Format todays date as ISO format
today = pd.Timestamp("2023-05-05", tz="America/New_York").isoformat()

# Set the tickers
tickers = ['JPM','JNJ','MSFT','RIO','PFE']

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices
df_portfolio = api.get_bars(
    tickers,
    timeframe,
    start = today,
    end = today
).df

In [7]:
# Reorganize the DataFrame
# Separate ticker data
JPM = df_portfolio[df_portfolio['symbol']=='JPM'].drop('symbol', axis=1)
JNJ = df_portfolio[df_portfolio['symbol']=='JNJ'].drop('symbol', axis=1)
MSFT = df_portfolio[df_portfolio['symbol']=='MSFT'].drop('symbol', axis=1)
RIO = df_portfolio[df_portfolio['symbol']=='RIO'].drop('symbol', axis=1)
PFE = df_portfolio[df_portfolio['symbol']=='PFE'].drop('symbol', axis=1)
# Concatenate the ticker DataFrames
df_combined = pd.concat([JPM,JNJ,MSFT,RIO,PFE], axis=1, keys=['JPM','JNJ','MSFT','RIO','PFE'])
df_combined

JPM                                     \
                             open    high      low   close    volume   
timestamp                                                              
2023-05-05 04:00:00+00:00  136.44  137.88  135.912  136.74  11106638   

                                                      JNJ                  \
                          trade_count        vwap    open    high     low   
timestamp                                                                   
2023-05-05 04:00:00+00:00      128401  136.707656  162.84  163.13  161.69   

                           ...      RIO                           PFE        \
                           ...   volume trade_count       vwap   open  high   
timestamp                  ...                                                
2023-05-05 04:00:00+00:00  ...  3297822       32857  63.354188  38.38  38.5   

                                                                          
                             low  close    volume trade_count       vwap  
timestamp                                                                 
2023-05-05 04:00:00+00:00  38.09  38.49  18908137      116391  38.345047  

[1 rows x 35 columns]

In [8]:
# Pick JPM ,JNJ, MSFT ,RIO, and PFE close prices
jpm_close_price = df_combined["JPM"]["close"].values
jnj_close_price = df_combined["JNJ"]["close"].values
msft_close_price = df_combined["MSFT"]["close"].values
rio_close_price = df_combined["RIO"]["close"].values
pfe_close_price = df_combined["PFE"]["close"].values

In [9]:
# Print close prices
print(f"Current JPM closing price: ${jpm_close_price}")
print(f"Current JNJ closing price: ${jnj_close_price}")
print(f"Current MSFT closing price: ${msft_close_price}")
print(f"Current RIO closing price: ${rio_close_price}")
print(f"Current PFE closing price: ${pfe_close_price}")

Current JPM closing price: $[136.74]
Current JNJ closing price: $[162.68]
Current MSFT closing price: $[310.65]
Current RIO closing price: $[63.67]
Current PFE closing price: $[38.49]


### Planning

In [12]:
# Format start and end date as ISO format
start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-08-07", tz="America/New_York").isoformat()

In [25]:
# Get 5 years' worth of historical data 
ticker_data = api.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df
# Reorganize the DataFrame
# Separate ticker data
JPM_2 = ticker_data[ticker_data['symbol']=='JPM'].drop('symbol', axis=1)
JNJ_2 = ticker_data[ticker_data['symbol']=='JNJ'].drop('symbol', axis=1)
MSFT_2 = ticker_data[ticker_data['symbol']=='MSFT'].drop('symbol', axis=1)
RIO_2 = ticker_data[ticker_data['symbol']=='RIO'].drop('symbol', axis=1)
PFE_2 = ticker_data[ticker_data['symbol']=='PFE'].drop('symbol', axis=1)

JPM_2 = JPM_2['close']
JNJ_2 = JNJ_2['close']
MSFT_2 = MSFT_2['close']
RIO_2 = RIO_2['close']
PFE_2 = PFE_2['close']


JPM_returns = JPM_2.pct_change().dropna().copy()
JNJ_returns = JNJ_2.pct_change().dropna().copy()
MSFT_returns = MSFT_2.pct_change().dropna().copy()
RIO_returns = RIO_2.pct_change().dropna().copy()
PFE_returns = PFE_2.pct_change().dropna().copy()

In [27]:
# Concatenate the ticker DataFrames
df_stock_data = pd.concat([JPM_returns,JNJ_returns,MSFT_returns,RIO_returns,PFE_returns], 
                          axis=1, keys=['JPM_2','JNJ_2','MSFT_2','RIO_2','PFE_2'])

# Display sample data
df_stock_data.head()

,JPM_2,JNJ_2,MSFT_2,RIO_2,PFE_2
timestamp,,,,,
2020-08-10 04:00:00+00:00,0.012679,-0.003836,-0.019908,0.010829,-0.001560
2020-08-11 04:00:00+00:00,0.031598,-0.007161,-0.023385,-0.004220,-0.015629
2020-08-12 04:00:00+00:00,-0.008476,0.018303,0.028567,0.015811,0.014289
2020-08-13 04:00:00+00:00,-0.005537,-0.011092,-0.002342,-0.008344,-0.004174
2020-08-14 04:00:00+00:00,0.000391,0.001622,0.000958,-0.003398,-0.002882
